In [ ]:
!pip install opencv-python mediapipe scikit-learn matplotlib numpy 


In [ ]:
!pip install tensorflow 

In [ ]:
import cv2
import numpy as np
import os
# from matplotlib import pyplot as plt
import mediapipe as mp
import time


In [ ]:
mp_holistic= mp.solutions.holistic      #Detection model
mp_drawing= mp.solutions.drawing_utils  #Drawing connections

In [ ]:
def mediapipe_detection(image, model):
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   #Color conversion
    image.flags.writeable= False                    #Image no longer writeable
    results= model.process(image)                   #Make prediction  
    image.flags.writeable= True                     #Image now writeable
    image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)   #Color conversion 
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,mp_holistic.FACEMESH_CONTOURS)      #Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)       #Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)  #Draw left hand Connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS) #Draw right hand Connections

In [ ]:
# cap = cv2.VideoCapture(0)

# #Access Mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():
        
#         #read feed from cam(0)
#         ret, frame= cap.read()
        
#         #make detection
#         image, results = mediapipe_detection(frame, holistic)
        
#         #Draw connections on the frame
#         draw_landmarks(image, results)
        
#         #Output connection to the screen
#         cv2.imshow('OpenCV Feed', image)
        
#         #Exit Program with 'q'
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
# cap.release()
# cv2.destroyAllWindows()
# cv2.waitKey(1)

In [ ]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [ ]:
#Path for exported data, numpy arrays
DATA_PATH= os.path.join('MP_DATA')

#Actions to detect
words = np.array(['hi','thanks','iloveyou', 'my', 'name', 'namaste','y','a','j','u'])
# alphabets= np.array(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'])
# numbers = np.arange(1,11)

# actions=np.concatenate([words,alphabets,numbers])
actions=words


#Thirty videos worth of data
no_sequences= 30

#Video length in frames
sequence_length= 30


In [ ]:
#Make folders for data collections
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [ ]:
# cap = cv2.VideoCapture(0)

# #Access Mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
#     #Loop through actions
#     for action in actions:
#         #Loop through videos
#         for sequence in range(no_sequences):
#             #Loop through frames  
#             for frame_num in range(sequence_length):
#                  #read feed from cam(0)
#                  ret, frame= cap.read()
                
#                  #make detection
#                  image, results = mediapipe_detection(frame, holistic)
                
#                  #Draw connections on the frame
#                  draw_landmarks(image, results)
                 
#                  #Apply wait logic:
#                  if frame_num== 0:
#                      cv2.putText(image, 'STARTING COLLECTION', (120, 200),
#                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (132, 63, 195), 4, cv2.LINE_AA)
#                      cv2.putText(image, 'Collecting frames for {} Video Number{}'.format(action, sequence), (15, 50),
#                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (130, 99, 0), 4, cv2.LINE_AA)
#                      cv2.waitKey(1000)
#                  else:
#                     cv2.putText(image, 'Collecting frames for {} Video Number{}'.format(action, sequence), (15, 50),
#                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (130, 99, 0), 4, cv2.LINE_AA)
                     
#                  #New Export keypoints
#                  keypoints= extract_keypoints(results)
#                  npy_path= os.path.join(DATA_PATH, action,str(sequence),str(frame_num))
#                  np.save(npy_path, keypoints)
                
#                  #Output connection to the screen
#                  cv2.imshow('OpenCV Feed', image)
                
#                  #Exit Program with 'q'
#                  if cv2.waitKey(10) & 0xFF == ord('q'):
#                      break 
#                  if cv2.waitKey(10) & 0xFF == ord('p'):
#                      cv2.putText(image, 'PAUSE', (120, 200),
#                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (132, 63, 195), 4, cv2.LINE_AA)
#                      cv2.waitKey(120000) 
        
       
# cap.release()
# cv2.destroyAllWindows()
# cv2.waitKey(1)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical       #if on silicon mac make a miniforge environment for tensorflow

label_map={label:num for num,label in enumerate(actions)}

In [ ]:
sequences,labels=[], []
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])     

In [ ]:
X= np.array(sequences)
y= to_categorical(labels).astype(int)

In [ ]:
X_train,X_test, y_train, y_test=train_test_split(X,y, test_size=0.05)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback=TensorBoard(log_dir=log_dir)


In [ ]:
model =Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128,return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train,y_train, epochs=250, callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
res=model.predict(X_test)
actions[np.argmax(res[2])]

In [ ]:
actions[np.argmax(y_test[2])]

In [ ]:
sequence=[]
sentence=[]
threshold=0.4

cap = cv2.VideoCapture(0)

#Access Mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
    while cap.isOpened():
        
        #read feed from cam(0)
        ret, frame= cap.read()
        
        #make detection
        image, results = mediapipe_detection(frame, holistic)
        
        #Draw connections on the frame
        draw_landmarks(image, results)
        
        #1.prediction Logic
        keypoints= extract_keypoints(results)
        sequence.append(keypoints)
        sequence= sequence[-30:]
        
        if len(sequence)== 30:
            res= model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
        
        #Output connection to the screen
        cv2.imshow('OpenCV Feed', image)
        
        #Exit Program with 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)